# Exp P: Architecture Universality Test

## 目的
複数のアーキテクチャでDual-Gradient Learningの相構造が再現されるかを検証し、
発見がアーキテクチャ非依存であることを確認する。

## 実験設計
- **モデル**: ResNet-18, ResNet-34, VGG-11, SimpleCNN（4種）
- **λ**: 0.15, 0.25, 0.35, 0.45, 0.55, 0.65（6点、相構造を横断）
- **η（ノイズ率）**: 0.4（固定）
- **Seeds**: 0, 1, 2（3個）
- **Total**: 4 × 6 × 3 = **72 runs**

## 期待される結果
- 全アーキテクチャで類似の相構造（ordered → critical → collapse）
- 臨界点λ_cがアーキテクチャ間で類似

## 推定時間
~72 runs × 12 min ≈ **14 時間**

In [ ]:
# ===== セットアップ =====
from google.colab import drive
drive.mount('/content/drive')

import os
from datetime import datetime

EXP_NAME = 'exp_P_architecture_universality'
TIMESTAMP = datetime.now().strftime('%Y%m%d_%H%M%S')
BASE_DIR = '/content/drive/MyDrive/dual-gradient-learning/Paper-A'
SAVE_DIR = f'{BASE_DIR}/{EXP_NAME}_{TIMESTAMP}'
os.makedirs(SAVE_DIR, exist_ok=True)
os.makedirs(f'{SAVE_DIR}/figures', exist_ok=True)

print(f'Experiment: {EXP_NAME}')
print(f'Timestamp: {TIMESTAMP}')
print(f'Save directory: {SAVE_DIR}')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils import parameters_to_vector
import torchvision
import torchvision.transforms as transforms
import numpy as np
import json
import time

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if torch.cuda.is_available():
    torch.backends.cudnn.benchmark = True
    print(f'GPU: {torch.cuda.get_device_name(0)}')
print(f'Device: {device}')

In [ ]:
# ===== モデル定義 =====
class IndexedDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
    def __getitem__(self, idx):
        img, label = self.dataset[idx]
        return img, label, idx
    def __len__(self):
        return len(self.dataset)

# ===== ResNet Block =====
class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, in_planes, planes, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, 3, stride, 1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, 3, 1, 1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, 1, stride, bias=False),
                nn.BatchNorm2d(planes))
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        return F.relu(out)

# ===== ResNet-18 =====
class ResNet18(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.in_planes = 64
        self.conv1 = nn.Conv2d(3, 64, 3, 1, 1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(64, 2, 1)
        self.layer2 = self._make_layer(128, 2, 2)
        self.layer3 = self._make_layer(256, 2, 2)
        self.layer4 = self._make_layer(512, 2, 2)
        self.linear = nn.Linear(512, num_classes)
    def _make_layer(self, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for s in strides:
            layers.append(BasicBlock(self.in_planes, planes, s))
            self.in_planes = planes
        return nn.Sequential(*layers)
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.adaptive_avg_pool2d(out, 1)
        out = out.view(out.size(0), -1)
        return self.linear(out)

# ===== ResNet-34 =====
class ResNet34(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.in_planes = 64
        self.conv1 = nn.Conv2d(3, 64, 3, 1, 1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(64, 3, 1)   # 3 blocks
        self.layer2 = self._make_layer(128, 4, 2)  # 4 blocks
        self.layer3 = self._make_layer(256, 6, 2)  # 6 blocks
        self.layer4 = self._make_layer(512, 3, 2)  # 3 blocks
        self.linear = nn.Linear(512, num_classes)
    def _make_layer(self, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for s in strides:
            layers.append(BasicBlock(self.in_planes, planes, s))
            self.in_planes = planes
        return nn.Sequential(*layers)
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.adaptive_avg_pool2d(out, 1)
        out = out.view(out.size(0), -1)
        return self.linear(out)

# ===== VGG-11 =====
class VGG11(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1), nn.BatchNorm2d(64), nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, 3, 1, 1), nn.BatchNorm2d(128), nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 256, 3, 1, 1), nn.BatchNorm2d(256), nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, 3, 1, 1), nn.BatchNorm2d(256), nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(256, 512, 3, 1, 1), nn.BatchNorm2d(512), nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, 1, 1), nn.BatchNorm2d(512), nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(512, 512, 3, 1, 1), nn.BatchNorm2d(512), nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, 1, 1), nn.BatchNorm2d(512), nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
        )
        self.classifier = nn.Linear(512, num_classes)
    def forward(self, x):
        out = self.features(x)
        out = out.view(out.size(0), -1)
        return self.classifier(out)

# ===== SimpleCNN =====
class SimpleCNN(nn.Module):
    """シンプルな4層CNN（ResNetやVGGより小さい）"""
    def __init__(self, num_classes=10):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, 1, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1, 1)
        self.conv3 = nn.Conv2d(64, 128, 3, 1, 1)
        self.conv4 = nn.Conv2d(128, 256, 3, 1, 1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc = nn.Linear(256 * 2 * 2, num_classes)
        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)
        self.bn4 = nn.BatchNorm2d(256)
    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))  # 16x16
        x = self.pool(F.relu(self.bn2(self.conv2(x))))  # 8x8
        x = self.pool(F.relu(self.bn3(self.conv3(x))))  # 4x4
        x = self.pool(F.relu(self.bn4(self.conv4(x))))  # 2x2
        x = x.view(x.size(0), -1)
        return self.fc(x)

# ===== モデルファクトリ =====
def get_model(model_name, num_classes=10):
    models = {
        'ResNet18': ResNet18,
        'ResNet34': ResNet34,
        'VGG11': VGG11,
        'SimpleCNN': SimpleCNN
    }
    return models[model_name](num_classes=num_classes)

In [ ]:
# ===== 実験パラメータ =====
BATCH_SIZE = 256
NUM_WORKERS = 4
EPOCHS = 100
LR = 0.1
K = 16

MODELS = ['ResNet18', 'ResNet34', 'VGG11', 'SimpleCNN']
LAMBDAS = [0.15, 0.25, 0.35, 0.45, 0.55, 0.65]
NOISE_RATE = 0.4  # 固定
SEEDS = [0, 1, 2]

# 実験条件リスト生成
experiments = []
for model_name in MODELS:
    for lam in LAMBDAS:
        for seed in SEEDS:
            experiments.append({
                'model': model_name,
                'lambda': lam,
                'seed': seed
            })

total_runs = len(experiments)
print(f'Models: {MODELS}')
print(f'Lambdas: {LAMBDAS}')
print(f'Noise rate: {NOISE_RATE} (fixed)')
print(f'Seeds: {SEEDS}')
print(f'Total runs: {total_runs}')
print(f'Estimated time: {total_runs * 12 / 60:.1f} hours')

# config保存
config = {
    'experiment': EXP_NAME,
    'timestamp': TIMESTAMP,
    'description': 'Architecture universality test',
    'parameters': {
        'models': MODELS,
        'lambdas': LAMBDAS,
        'noise_rate': NOISE_RATE,
        'seeds': SEEDS,
        'epochs': EPOCHS,
        'batch_size': BATCH_SIZE,
        'lr': LR,
        'K': K
    },
    'total_runs': total_runs
}
with open(f'{SAVE_DIR}/exp_P_config.json', 'w') as f:
    json.dump(config, f, indent=2)
print(f'Config saved')

In [ ]:
# ===== ユーティリティ関数 =====
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)

def load_cifar10():
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
    testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
    return trainset, testset

def get_data_loaders(trainset, testset):
    indexed_trainset = IndexedDataset(trainset)
    train_loader = DataLoader(indexed_trainset, batch_size=BATCH_SIZE, shuffle=True,
                              num_workers=NUM_WORKERS, pin_memory=True, persistent_workers=True, drop_last=True)
    test_loader = DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False,
                             num_workers=NUM_WORKERS, pin_memory=True, persistent_workers=True)
    return train_loader, test_loader

def inject_noise(labels, noise_rate, seed):
    np.random.seed(seed + 1000)
    noisy_labels = labels.copy()
    n_noisy = int(noise_rate * len(labels))
    noisy_indices = np.random.choice(len(labels), n_noisy, replace=False)
    for idx in noisy_indices:
        choices = [i for i in range(10) if i != labels[idx]]
        noisy_labels[idx] = np.random.choice(choices)
    return noisy_labels

def evaluate(model, test_loader):
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device, non_blocking=True), targets.to(device, non_blocking=True)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    return correct / total

In [ ]:
# ===== Dual-Gradient Learning =====
def train_dual_gradient(model, train_loader, test_loader, clean_labels, noisy_labels, lam):
    optimizer = optim.SGD(model.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4)
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[50, 75], gamma=0.1)
    criterion = nn.CrossEntropyLoss()
    
    clean_labels_tensor = torch.tensor(clean_labels, device=device)
    noisy_labels_tensor = torch.tensor(noisy_labels, device=device)
    
    cached_g_value = None
    global_step = 0
    
    best_acc = 0
    cos_history = []
    
    for epoch in range(EPOCHS):
        model.train()
        epoch_cos = []
        
        for inputs, _, indices in train_loader:
            inputs = inputs.to(device, non_blocking=True)
            indices = indices.to(device, non_blocking=True)
            batch_noisy = noisy_labels_tensor[indices]
            batch_clean = clean_labels_tensor[indices]
            
            # Structure gradient
            optimizer.zero_grad()
            outputs = model(inputs)
            loss_struct = criterion(outputs, batch_noisy)
            loss_struct.backward(retain_graph=True)
            g_struct = parameters_to_vector([p.grad for p in model.parameters()]).clone()
            
            # Value gradient
            if global_step % K == 0 or cached_g_value is None:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss_value = criterion(outputs, batch_clean)
                loss_value.backward()
                cached_g_value = parameters_to_vector([p.grad for p in model.parameters()]).clone()
            
            # Normalize and mix
            g_struct_norm = g_struct / (g_struct.norm() + 1e-12)
            g_value_norm = cached_g_value / (cached_g_value.norm() + 1e-12)
            
            cos_sim = (g_struct_norm @ g_value_norm).item()
            epoch_cos.append(cos_sim)
            
            g_mix = (1 - lam) * g_struct_norm + lam * g_value_norm
            
            optimizer.zero_grad()
            idx = 0
            for p in model.parameters():
                numel = p.numel()
                p.grad = g_mix[idx:idx+numel].view(p.shape).clone()
                idx += numel
            optimizer.step()
            global_step += 1
        
        scheduler.step()
        cos_history.append(np.mean(epoch_cos))
        
        if (epoch + 1) % 10 == 0:
            acc = evaluate(model, test_loader)
            best_acc = max(best_acc, acc)
    
    final_acc = evaluate(model, test_loader)
    avg_cos = np.mean(cos_history)
    
    return final_acc, max(best_acc, final_acc), avg_cos

In [ ]:
# ===== データ準備 =====
trainset, testset = load_cifar10()
clean_labels = np.array(trainset.targets)
train_loader, test_loader = get_data_loaders(trainset, testset)

print('Data prepared')
print('Warmup complete.')

In [ ]:
# ===== メイン実験ループ =====
results = []
checkpoint_file = f'{SAVE_DIR}/exp_P_checkpoint.json'
completed = set()

if os.path.exists(checkpoint_file):
    with open(checkpoint_file, 'r') as f:
        results = json.load(f)
    for r in results:
        completed.add((r['model'], r['lambda'], r['seed']))
    print(f'Checkpoint loaded: {len(completed)} runs completed')

# ノイズラベル生成（実験間で共通）
noisy_labels_cache = {}
for seed in SEEDS:
    noisy_labels_cache[seed] = inject_noise(clean_labels, NOISE_RATE, seed)

run_counter = 0
exp_start = time.time()

for exp in experiments:
    run_counter += 1
    model_name = exp['model']
    lam = exp['lambda']
    seed = exp['seed']
    
    key = (model_name, lam, seed)
    if key in completed:
        continue
    
    print(f'\n[{run_counter}/{total_runs}] {model_name} λ={lam:.2f} seed={seed}')
    t0 = time.time()
    
    set_seed(seed)
    noisy_labels = noisy_labels_cache[seed]
    model = get_model(model_name).to(device)
    
    # パラメータ数を表示（最初の1回だけ）
    if seed == 0 and lam == LAMBDAS[0]:
        n_params = sum(p.numel() for p in model.parameters())
        print(f'  Parameters: {n_params:,}')
    
    final_acc, best_acc, avg_cos = train_dual_gradient(
        model, train_loader, test_loader,
        clean_labels, noisy_labels, lam
    )
    elapsed = time.time() - t0
    
    result = {
        'experiment_id': f'P-{run_counter:03d}',
        'experiment': EXP_NAME,
        'model': model_name,
        'lambda': lam,
        'noise_rate': NOISE_RATE,
        'seed': seed,
        'test_acc': final_acc,
        'test_error': 1 - final_acc,
        'best_acc': best_acc,
        'best_error': 1 - best_acc,
        'avg_cos_struct_value': avg_cos,
        'time_seconds': elapsed
    }
    results.append(result)
    
    with open(checkpoint_file, 'w') as f:
        json.dump(results, f, indent=2)
    
    status = ' ⚠️ COLLAPSE' if (1 - final_acc) > 0.5 else (' ✅ EXCELLENT' if (1 - final_acc) < 0.15 else '')
    print(f'  Error: {1-final_acc:.4f} | cos: {avg_cos:.4f} | Time: {elapsed/60:.1f} min{status}')
    
    completed_count = len(results)
    avg_time = sum([r['time_seconds'] for r in results]) / completed_count
    remaining = total_runs - completed_count
    eta_hours = (remaining * avg_time) / 3600
    print(f'  Progress: {completed_count}/{total_runs} | ETA: {eta_hours:.1f} hours')

print(f'\n{"="*60}')
print(f'EXPERIMENT P COMPLETE')
print(f'Total time: {(time.time()-exp_start)/3600:.2f} hours')
print(f'{"="*60}')

In [ ]:
# ===== 結果保存 =====
import pandas as pd

with open(f'{SAVE_DIR}/exp_P_results.json', 'w') as f:
    json.dump(results, f, indent=2)

df = pd.DataFrame(results)
df.to_csv(f'{SAVE_DIR}/exp_P_results.csv', index=False)

print(f'Results saved to {SAVE_DIR}/')

In [ ]:
# ===== 可視化: アーキテクチャ比較 =====
import matplotlib.pyplot as plt
import pandas as pd

df = pd.DataFrame(results)

fig, ax = plt.subplots(figsize=(10, 6))

colors = {'ResNet18': 'C0', 'ResNet34': 'C1', 'VGG11': 'C2', 'SimpleCNN': 'C3'}

for model_name in MODELS:
    df_m = df[df['model'] == model_name]
    stats = df_m.groupby('lambda')['test_error'].agg(['mean', 'std'])
    
    ax.errorbar(stats.index, stats['mean'], yerr=stats['std'],
                marker='o', capsize=4, linewidth=2, markersize=8,
                label=model_name, color=colors[model_name])

ax.set_xlabel('λ', fontsize=12)
ax.set_ylabel('Test Error', fontsize=12)
ax.set_title('Architecture Universality (η=40%)', fontsize=13, fontweight='bold')
ax.legend()
ax.grid(alpha=0.3)
ax.set_xlim([0.10, 0.70])

plt.tight_layout()
plt.savefig(f'{SAVE_DIR}/figures/exp_P_architecture_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# ===== 臨界点比較 =====
print('\n' + '='*70)
print('CRITICAL POINT COMPARISON')
print('='*70)

for model_name in MODELS:
    df_m = df[df['model'] == model_name]
    stats = df_m.groupby('lambda')['test_error'].agg(['mean', 'std'])
    
    best_lambda = stats['mean'].idxmin()
    critical_lambda = stats['std'].idxmax()
    
    print(f'\n{model_name}:')
    print(f'  Best λ: {best_lambda} (error: {stats["mean"].min():.4f})')
    print(f'  Critical λ (max variance): {critical_lambda}')

In [ ]:
# ===== サマリー =====
print('\n' + '='*70)
print('EXPERIMENT P: ARCHITECTURE UNIVERSALITY - SUMMARY')
print('='*70)

# 全アーキテクチャの臨界点を比較
critical_points = []
for model_name in MODELS:
    df_m = df[df['model'] == model_name]
    stats = df_m.groupby('lambda')['test_error'].agg(['std'])
    critical_lambda = stats['std'].idxmax()
    critical_points.append(critical_lambda)

print(f'\nCritical points: {critical_points}')
print(f'Mean: {np.mean(critical_points):.2f}')
print(f'Std: {np.std(critical_points):.2f}')

if np.std(critical_points) < 0.1:
    print('\n✅ UNIVERSALITY CONFIRMED: Critical points are consistent across architectures')
else:
    print('\n⚠️ WARNING: Critical points vary significantly across architectures')

print(f'\nSave directory: {SAVE_DIR}')